### Class Decorators

Let's come back to decorators.

So far, we have been using decorators to decorate functions - but of course, we could also use them to decorate classes:

Let's start with a simple example first, like we saw in the lecture:

In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls
    
def checking(cls):
    cls.account_type = 'checking'
    return cls

In [2]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

And if we inspect our classes, we'll see that the `account_type` attribute has been injected by the decorator:

In [3]:
Bank1Savings.account_type, Bank1Checking.account_type

('savings', 'checking')

Of course, we could make even this simple example a little DRYer, by making a parameterized decorator:

In [4]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [5]:
@account_type('Savings')
class Bank1Savings:
    pass

@account_type('Checking')
class Bank1Checking:
    pass

In [6]:
Bank1Savings.account_type, Bank1Checking.account_type

('Savings', 'Checking')

We're not restricted to just adding data attributes either.

Let's create a class decorator to inject a new function into the class before we return it:

In [7]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls

In [8]:
@hello
class Person:
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return self.name

In [9]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

As you can see, the `Person` class now has an attribute `hello` which is a function.

So, it will then become a bound method when we call it from an instance of `Person`:

In [10]:
p = Person('Guido')

In [11]:
p.hello()

'Guido says hello!'

These examples are simple enough to understand what's going on, but not very useful.

But we can do some interesting things.

For example, suppose we want to log every call to every callable in some class.

We could certainly do it this way:

In [12]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner    

In [13]:
class Person:
    @func_logger
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    @func_logger
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

In [14]:
p = Person('John', 78)

log: Person.__init__((<__main__.Person object at 0x7fad8914b3d0>, 'John', 78), {}) = None


In [15]:
p.greet()

log: Person.greet((<__main__.Person object at 0x7fad8914b3d0>,), {}) = Hello, my name is John and I am 78


'Hello, my name is John and I am 78'

But this is kind of tedious if we have many methods in our class. Not very DRY!

Instead, how about creating a class decorator that will decorate every callable in a given class with the logger decorator:

In [16]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

So now we could do this:

In [17]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating: <class '__main__.Person'> __init__
decorating: <class '__main__.Person'> greet


In [18]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              'greet': <function __main__.Person.greet(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [19]:
p = Person('John', 78)

log: Person.__init__((<__main__.Person object at 0x7fad6820ca10>, 'John', 78), {}) = None


In [20]:
p.greet()

log: Person.greet((<__main__.Person object at 0x7fad6820ca10>,), {}) = Hello, my name is John and I am 78


'Hello, my name is John and I am 78'

Now we have to be a bit careful. Although this class decorator seems to work fine, it will have issues with static and class methods!

In [21]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print('static_method invoked...')
    
    @classmethod
    def cls_method(cls):
        print(f'cls_method invoked for {cls}...')
        
    def instance_method(self):
        print(f'instance_method invoked for {self}')

decorating: <class '__main__.Person'> instance_method


In [22]:
Person.static_method()

static_method invoked...


In [23]:
Person.cls_method()

cls_method invoked for <class '__main__.Person'>...


In [24]:
Person().instance_method()

instance_method invoked for <__main__.Person object at 0x7fad8914bd50>
log: Person.instance_method((<__main__.Person object at 0x7fad8914bd50>,), {}) = None


You'll notice that in the `cls_method` and `instance_method` cases, the logger printout never showed up! In fact, we did not get the message that these methods had been decorated.

What happened?

The problem is that static and class methods are not functions, they are actually descriptors, not callables.

In [25]:
class Person:
    @staticmethod
    def static_method():
        pass

In [26]:
Person.__dict__['static_method']

In [27]:
callable(Person.__dict__['static_method'])

False

So, they were not decorated at all.

Which is probably a good thing, because our decorator is expecting to decorate a function, not a class!

This, by the way, is why when you decorate static or class methods using a function decorator in your classes, you should do so before you decorate it with the `@staticmethod` or `@classmethod` decorators:

In [28]:
class Person:
    @staticmethod
    @func_logger
    def static_method():
        pass

In [29]:
Person.static_method()

log: Person.static_method((), {}) = None


But if you try it this way around, things aren't so happy:

In [30]:
class Person:
    @func_logger
    @staticmethod
    def static_method():
        pass

In [31]:
Person.static_method()

TypeError: 'staticmethod' object is not callable

We can actually fix this problem in our class decorator if we really wanted to.

Let's first examine two things separately.

First let's make sure we can recognize the type of a class or static method in our class:

In [32]:
class Person:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def class_method(cls):
        pass

In [33]:
type(Person.__dict__['static_method'])

staticmethod

In [34]:
type(Person.__dict__['class_method'])

classmethod

Next, can we somehow get back to the original function that was wrapped by the `@staticmethod` and `@classmethod` decorators?

The answer is yes, since these are method objects - we've seen this before when we studied the relationship between functions and descriptors and how methods were created.

In [35]:
Person.__dict__['static_method'].__func__

<function __main__.Person.static_method()>

In [36]:
Person.__dict__['class_method'].__func__

<function __main__.Person.class_method(cls)>

So now, we could modify our class decorator needs to unwrap any class or static methods, decorate the original function, and then re-wrap it with the appropriate `classmethod` or `instancemethod` decorator:

In [37]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
    return cls

In [38]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print('static_method called...', a, b)
        
    @classmethod
    def class_method(cls, a, b):
        print('class_method called...', cls, a, b)
        
    def instance_method(self, a, b):
        print('instance_method called...', self, a, b)

decorating static method <function Person.static_method at 0x7fad8914a680>
<staticmethod object at 0x7fad6822af10> <class 'staticmethod'>
decorating class method <function Person.class_method at 0x7fad8914ab00>
decorating: <class '__main__.Person'> instance_method


In [39]:
Person.static_method(10, 20)

static_method called... 10 20
log: Person.static_method((10, 20), {}) = None


In [40]:
Person.class_method(10, 20)

class_method called... <class '__main__.Person'> 10 20
log: Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [41]:
Person().instance_method(10, 20)

instance_method called... <__main__.Person object at 0x7fad98071950> 10 20
log: Person.instance_method((<__main__.Person object at 0x7fad98071950>, 10, 20), {}) = None


Not bad... Not what about properties?

In [42]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating: <class '__main__.Person'> __init__


Hmm, the property was not decorated...

Let's see what the type of that property is (you should already know this):

In [43]:
type(Person.__dict__['name'])

property

In [44]:
isinstance(Person.__dict__['name'], property)

True

And how do we get the original functions on a property?

In [45]:
prop = Person.__dict__['name']

In [46]:
prop.fget

<function __main__.Person.name(self)>

In [47]:
prop.fset, prop.fdel

(None, None)

Hmm, so maybe we can decorate the `fget`, `fset`, and `fdel` functions of the property (if they are not `None`).

We can't just replace the functions, because `fget`, `fset` and `fdel` are actually read-only properties themselves, that return the original functions. But we could create a new property based off thge original one, substituting our decorated getter, setter and deleter.

Recall that the `getter()`, `setter()` and `deleter()` methods are methods that will create a copy of the original property, but substitute the `fget`, `fset` and `fdel` methods (that's how these are used as decorators).

In [48]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
    return cls

In [49]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating: <class '__main__.Person'> __init__
decorating property <property object at 0x7fad89142ad0>


In [50]:
p = Person('David')

log: Person.__init__((<__main__.Person object at 0x7fad6821db90>, 'David'), {}) = None


In [51]:
p.name

log: Person.name((<__main__.Person object at 0x7fad6821db90>,), {}) = David


'David'

Ha!! Pretty cool...

Let's make sure it works if we have setters and deleters as well:

In [52]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value
        
    @name.deleter
    def name(self):
        print('deleting name...')

decorating: <class '__main__.Person'> __init__
decorating property <property object at 0x7fad68232bf0>


In [53]:
p = Person('David')

log: Person.__init__((<__main__.Person object at 0x7fad6821d7d0>, 'David'), {}) = None


In [54]:
p.name

log: Person.name((<__main__.Person object at 0x7fad6821d7d0>,), {}) = David


'David'

In [55]:
p.name = 'Beazley'

log: Person.name((<__main__.Person object at 0x7fad6821d7d0>, 'Beazley'), {}) = None


In [56]:
del p.name

deleting name...
log: Person.name((<__main__.Person object at 0x7fad6821d7d0>,), {}) = None


Success!!

A bit mind-bending, but nonetheless, cool stuff!

Still, this is not perfect... :(

We can still run into trouble because not every callable is a function that can be decorated:

In [57]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print('called instance of Other...')
            
    other = Other()

decorating: <class '__main__.Person'> Other
decorating: <class '__main__.Person'> other


So, as you see it decorated both the class `Other` (since classes are callables), and it decorated `other` since we made instances of `Other` callable too.

How does that work with the logger though:

In [58]:
Person.Other

<function __main__.Person.Other()>

In [59]:
Person.other

<function __main__.func_logger.<locals>.inner()>

And that's the problem, because `Other` and `other` are callables, they have been replaced in our class by what comes out of the decorator - a function.

So maybe we can use the `inspect` module to restrict our callables further:

In [60]:
import inspect

In [61]:
class MyClass:
    @staticmethod
    def static_method():
        pass
    
    @classmethod
    def cls_method(cls):
        pass
    
    def inst_method(self):
        pass
    
    @property
    def name(self):
        pass
    
    def __add__(self, other):
        pass
    
    class Other:
        def __call__(self):
            pass
        
    other = Other()
    

In [62]:
keys = ('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')
inspect_funcs = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor')

In [63]:
print(keys)

('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')


In [64]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False        


As you can see we could use inspect to only pick things that are routines instead of more general callables. Properties, static and class methods we are already handling specially, so I'm going to move the callable check last in the `if...elif` block so we handle static and class methods first (since they are classified as routines too).

In [65]:
import inspect

def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [66]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()
    

decorating static method <function MyClass.static_method at 0x7fad6821c0e0>
decorating class method <function MyClass.cls_method at 0x7fad6821c320>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x7fad6820f5f0>
decorating: <class '__main__.MyClass'> __add__


In [67]:
MyClass.Other, MyClass.other

(__main__.MyClass.Other, <__main__.MyClass.Other at 0x7fad6828bb10>)

In [68]:
MyClass.other()

<__main__.MyClass.Other object at 0x7fad6828bb10>.__call__ called...


No log, that was expected.

In [69]:
MyClass.static_method()

static_method called...
log: MyClass.static_method((), {}) = None


In [70]:
MyClass.cls_method()

class method called...
log: MyClass.cls_method((<class '__main__.MyClass'>,), {}) = None


In [71]:
MyClass().inst_method()

instance method called...
log: MyClass.inst_method((<__main__.MyClass object at 0x7fad682c19d0>,), {}) = None


In [72]:
MyClass().name

name getter called...
log: MyClass.name((<__main__.MyClass object at 0x7fad6828b650>,), {}) = None


In [73]:
MyClass() + MyClass()

__add__ called...
log: MyClass.__add__((<__main__.MyClass object at 0x7fad8914bb50>, <__main__.MyClass object at 0x7fad8914b5d0>), {}) = None


If we really wanted to, we could also decorate the `Other` class:

In [74]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    @class_logger
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()
    

decorating: <class '__main__.MyClass.Other'> __call__
decorating static method <function MyClass.static_method at 0x7fad68239b90>
decorating class method <function MyClass.cls_method at 0x7fad68239170>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x7fad682185f0>
decorating: <class '__main__.MyClass'> __add__


In [75]:
MyClass.other()

<__main__.MyClass.Other object at 0x7fad8912f3d0>.__call__ called...
log: MyClass.Other.__call__((<__main__.MyClass.Other object at 0x7fad8912f3d0>,), {}) = None


We could also do a bit of DRYing on our decorator code.

First let's handle the static and class methods:

In [76]:
import inspect

def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod) or isinstance(obj, classmethod):
            type_ = type(obj)
            original_func = obj.__func__
            print(f'decorating {type_.__name__} method', original_func)
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [77]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    @class_logger
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()

decorating: <class '__main__.MyClass.Other'> __call__
decorating staticmethod method <function MyClass.static_method at 0x7fad6820bb00>
decorating classmethod method <function MyClass.cls_method at 0x7fad6820b0e0>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x7fad68218e30>
decorating: <class '__main__.MyClass'> __add__


In [78]:
MyClass.static_method()

static_method called...
log: MyClass.static_method((), {}) = None


In [79]:
MyClass.cls_method()

class method called...
log: MyClass.cls_method((<class '__main__.MyClass'>,), {}) = None


Finally, let's see if we can clean up the block to handle properties - I don't like these repeated nested if statements that basically do the almost same thing:

In [80]:
import inspect

def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, staticmethod) or isinstance(obj, classmethod):
            type_ = type(obj)
            original_func = obj.__func__
            print(f'decorating {type_.__name__} method', original_func)
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
        elif isinstance(obj, property):
            print('decorating property', obj)
            methods = (('fget', 'getter'), ('fset', 'setter'), ('fdel', 'deleter'))
            for prop, method in methods:
                if getattr(obj, prop):
                    obj = getattr(obj, method)(func_logger(getattr(obj, prop)))
            setattr(cls, name, obj)
        elif inspect.isroutine(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [81]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
        
    @name.setter
    def name(self, value):
        print('name setter called...')
        
    @name.deleter
    def name(self):
        print('name deleter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    @class_logger
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()

decorating: <class '__main__.MyClass.Other'> __call__
decorating staticmethod method <function MyClass.static_method at 0x7fad8914a170>
decorating classmethod method <function MyClass.cls_method at 0x7fad8914a3b0>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x7fad682be950>
decorating: <class '__main__.MyClass'> __add__


In [82]:
MyClass().name

name getter called...
log: MyClass.name((<__main__.MyClass object at 0x7fad98076fd0>,), {}) = None


In [83]:
MyClass().name = 'David'

name setter called...
log: MyClass.name((<__main__.MyClass object at 0x7fad980764d0>, 'David'), {}) = None


In [84]:
del MyClass().name

name deleter called...
log: MyClass.name((<__main__.MyClass object at 0x7fad98076410>,), {}) = None
